# Collaborative Filtering: pick the best rated unseen film from the most similar user

In [3]:
import pandas as pd
import numpy as np

In [4]:
movies_df = pd.read_csv('../data/ml-latest-small/movies.csv')

In [5]:
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv')
user_item_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def get_similar_users(user_id):
    user_correlation = user_correlation_matrix.iloc[user_correlation_matrix.index==user_id].T.sort_values(by=user_id, \
                                                        ascending=False).dropna()
    best_similar_users = user_correlation[user_correlation>0.2].dropna()[1:]
    return best_similar_users.index

In [13]:
user_correlation_matrix = user_item_matrix.T.corr()
user_correlation_matrix.tail()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
606,0.066378,0.583333,-0.791334,0.144603,0.244321,-0.049192,0.137771,0.253582,0.572700,-0.382955,...,0.290490,0.140613,0.318473,0.682949,0.167062,1.000000,0.114191,0.240842,0.533002,0.389185
607,0.174557,NaN,-0.333333,0.116518,0.231080,0.255639,0.402792,0.251280,NaN,-0.241121,...,0.698241,0.217210,0.192787,0.035806,-0.299641,0.114191,1.000000,0.200814,0.190117,0.106605
608,0.268070,-0.125000,-0.395092,-0.170501,-0.020546,0.125428,0.008081,0.434423,0.336625,-0.571043,...,0.473967,0.297646,0.086423,0.053986,-0.075673,0.240842,0.200814,1.000000,0.488929,0.147606
609,-0.175412,NaN,NaN,-0.277350,0.384111,0.193649,0.420288,0.141860,NaN,NaN,...,1.000000,0.188512,0.343303,0.641624,-0.550000,0.533002,0.190117,0.488929,1.000000,-0.521773
610,-0.032086,0.623288,0.569562,-0.043786,0.040582,0.115580,0.341233,0.167931,0.615638,-0.205081,...,0.007025,0.049263,0.270908,0.310611,0.462274,0.389185,0.106605,0.147606,-0.521773,1.000000


In [14]:
users_not_seen_films = user_item_matrix.T.isna() 

In [15]:
users_seen_films = user_item_matrix.T.notna()

In [16]:
def get_seen_films(user_id):
    user_seen_boolean_series = users_seen_films[[user_id]]
    seen_films = user_seen_boolean_series.index[user_seen_boolean_series[user_id]].tolist()
    return seen_films

In [17]:
def get_sorted_films_by_rating(user_id):
    output = user_item_matrix.loc[[user_id]].T.sort_values(by=user_id, ascending=False).dropna().index
    return output.tolist()

In [21]:
def suggest_best_film_from_best_similar_user(user_id, get_seen_films, get_similar_users, get_sorted_films_by_rating) : 
    
    seen_films = get_seen_films(user_id)    
    sorted_similar_users = get_similar_users(user_id)  
    
    next_user = sorted_similar_users[0]
    suggestion = get_sorted_films_by_rating(next_user)[0]    
    j = 0 
    i = 0
    
    while suggestion in seen_films:
        i+=1
        suggestion = get_sorted_films_by_rating(next_user)[i]
        if i == len(get_sorted_films_by_rating(next_user)) -1 :            
            j+=1
            next_user = sorted_similar_users[j]
            i = -1
    return suggestion

In [23]:
suggest_best_film_from_best_similar_user(5, get_seen_films, get_similar_users, get_sorted_films_by_rating)

350